## Data

In [ ]:
# Recimo da ze imamo slike za training in validation
import os

train_dir_name = "/image_slicing/train/"
validation_dir_name = "/image_slicing/validation/"

train_positive_dir = os.path.join(train_dir_name + "positive")
train_negative_dir = os.path.join(train_dir_name + "negative")

validation_positive_dir = os.path.join(validation_dir_name + "positive")
validation_negative_dir = os.path.join(validation_dir_name + "negative")

In [ ]:
# print total length of each dataset
print('total training positive images:', len(os.listdir(train_positive_dir)))
print('total training negative images:', len(os.listdir(train_negative_dir)))
print('total validation positive images:', len(os.listdir(validation_positive_dir)))
print('total validation negative images:', len(os.listdir(validation_negative_dir)))

In [ ]:
# preprocessing needs to be done:
preprocessing_needed = True

from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # All images will be rescaled by 1./255

if preprocessing_needed:
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir_name,
        # target_size=(200, 200), # to be discussed?
        batch_size=120,
        class_mode='binary')
    validation_generator = validation_datagen.flow_from_directory(
        validation_dir_name,
        # target_size=(200, 200), # to be discussed?
        batch_size=19,
        class_mode='binary',
        shuffle=False)

## Model

In [ ]:
# imports
import tensorflow as tf
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [ ]:
img_shape = (200, 200, 3) #??

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = img_shape),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])
print(model.summary())

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

TODO: Model checkpoints!! https://keras.io/api/callbacks/model_checkpoint/ 

In [ ]:

# BEFORE:
# history = model.fit(train_generator,
#       steps_per_epoch=8,
#       epochs=15,
#       verbose=1,
#       validation_data = validation_generator,
#       validation_steps=8)

# AFTER:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# Checkpoint directory
checkpoint_dir = "/image_slicing/checkpoints/"

# Create directory if it does not exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_path = "checkpoints/model_{epoch:02d}-{val_loss:.2f}.hdf5"

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,  # Save only the best model
                             monitor='val_loss',   # Monitor the validation loss
                             mode='min',           # Minimize the validation loss
                             verbose=1)            # Print out when a model is being saved

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=8,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator,
                    validation_steps=8,
                    callbacks=[checkpoint]) 

In [ ]:
# Sample evaluation before exit.
model.evaluate(validation_generator)